# Clip Tiles to County Boundary for Bulk Download

This notebook contains steps to clip the tile index - dem, aerial, point cloud - to an area of interest (AOI) boundary.
- Acquire the Tile Index Layer of interest
    - DEM
    - Aerial
    - Point Cloud
- Bring in your AOI
    - shapefile
    - GeoJson
    - Rest End Point
    - other format
- Spatial Analysis
- Export
    - List to work with locally
    - download files directly from S3

## Get Tile Index

Use Tile Index Geopackage link from the [README](../README.md#) or from [KyFromAbove Open Data Explorer](https://kyfromabove.s3.us-west-2.amazonaws.com/index.html).  Just navigate to the appropriate Aerial or Elevation Folder.<br>
<br><center>
<img src="https://github.com/ianhorn/kyfromabove-on-aws-examples/blob/main/media/aws_explorer.jpg?raw=true" width="600" height="200">
</center><br>

Once you navigate to the Geopackage Folder, you can right click on the Geopackage you need and copy the link.

In [9]:
# import modules
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import geopandas as gpd
# import matplotlib
import os

ModuleNotFoundError: No module named 'matplotlib'

## Read in the tile index

For Pandas GeodataFrames, I will use the prefix **gdf_**.

The only thing you need to change would be the URL to the tile index layer and your AOI url/path.

In [2]:
# enter the url for title_url
# you can copy and paste from above
tile_url = 'https://kyfromabove.s3-us-west-2.amazonaws.com/imagery/orthos/tile-grids/kyfromabove_phase3_aerial_5k_grid.gpkg'

In [ ]:
# read the tile_url into a geodataframe
gdf_tiles = gpd.read_file(tile_url)
gdf_tiles = gdf_tiles.to_crs(epsg=4326)
gdf_tiles.head(1)

f:\workspace_2\AWS\kyfromabove-on-aws-examples\venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: unable to open database file: this file is a WAL-enabled database. It cannot be opened because it is presumably read-only or in a read-only directory. Retrying with IMMUTABLE=YES open option
  return ogr_read(


,tilename,key,aws_url,size,geometry
0,N038E388,imagery/orthos/Phase3/KY_KYAPED_2022_Season2_3...,https://kyfromabove.s3.us-west-2.amazonaws.com...,1017406604,"MULTIPOLYGON (((-82.94931 38.74231, -82.95107 ..."


In [ ]:
# Plot the tiles, no need to add a map yet.
print(gdf_tiles.columns)
# gdf_tiles.plot()

Index(['tilename', 'key', 'aws_url', 'size', 'geometry'], dtype='object')


___
## Add an area of interest

This can be done by reading in a file locally or making a request.  For this example I leverage the KyFromAbove [Vector Services](https://kygisserver.ky.gov/arcgis) to query the [CountyShading](https://kygisserver.ky.gov/arcgis/rest/services/WGS84WM_Services/Ky_CountyShading_WGS84WM/MapServer) service for Fleming County. I clicked on the service, under layers click *County_Information (0)* to open up the layer  I will query.  Scroll to the bottom and click *query*.  

In the *Where* clause, use `1=1'` to get everything\*\*.  In *Out Fields*, enter _*_.  Switch the *Format* to `GeoJson`.  Press the `Query (Get)` button.  Copy the URL for the GeoJson.

_If you use chrome, the [Map-Services-Enhanced](https://github.com/raykendo/Map-Services-Enhanced) extension will let you copy a shortened url of your query._

\*_Note: If you have a geometry (Shapefile, GeoJson, GeoParquet, etc) file locally, you can substitute the `url` for your data file path. Geopandas can read many different formats._

\*\* _[ArcGIS Rest API Services](https://kygisserver.ky.gov/arcgis/sdk/rest/index.html#//02ss0000006v000000)_

In [6]:
# paste in the url from the rest endpoint query
aoi_url = 'https://kygisserver.ky.gov/arcgis/rest/services/WGS84WM_Services/Ky_CountyShading_WGS84WM/MapServer/0/query?where=1%3D1&units=esriSRUnit_Foot&outFields=*&returnExtentOnly=false&featureEncoding=esriDefault&f=geojson'

# use gdf for geodatframe
gdf_aoi = gpd.read_file(aoi_url)
gdf_aoi.head(3)

,OBJECTID,NAME,FIPS_ID,NAME2,ADDNAME,SEAT,SEAT2,FORM_YR,PARENTCO,FIPS_NO,...,LAST_UPDT,XY_SOURCE,XY_ISSUES,ATT_SOURCE,ATT_ISSUES,COMMENTS,KYTC_District_Num,Shape.area,Shape.len,geometry
0,1,ADAIR,21001,Adair,LAKE CUMBERLAND,Columbia,COLUMBIA,1802,Green,1,...,741484800000,BOUNDARY DIGITIZED FROM USGS 7.5 MINUTE TOPOGR...,None,SECRETARY OF STATE,None,None,8,1.149684e+10,462429.341380,"POLYGON ((-85.16518 37.3104, -85.16992 37.3120..."
1,2,ALLEN,21003,Allen,BARREN RIVER,Scottsville,SCOTTSVILLE,1815,"Barren, Warren",3,...,741484800000,BOUNDARY DIGITIZED FROM USGS 7.5 MINUTE TOPOGR...,None,SECRETARY OF STATE,None,None,3,9.811755e+09,557142.359439,"POLYGON ((-86.14067 36.91733, -86.14051 36.917..."
2,3,ANDERSON,21005,Anderson,BLUEGRASS,Lawrenceburg,LAWRENCEBURG,1827,"Franklin, Mercer, Washington",5,...,741484800000,BOUNDARY DIGITIZED FROM USGS 7.5 MINUTE TOPOGR...,None,SECRETARY OF STATE,None,None,7,5.693637e+09,483724.500281,"POLYGON ((-85.02371 38.12905, -85.02352 38.128..."


In [7]:
# print columns
gdf_aoi.columns

Index(['OBJECTID', 'NAME', 'FIPS_ID', 'NAME2', 'ADDNAME', 'SEAT', 'SEAT2',
       'FORM_YR', 'PARENTCO', 'FIPS_NO', 'FIPS_TXT', 'SMIS', 'SP_ZONE',
       'REGION', 'POP70', 'POP80', 'POP90', 'POP00', 'POP10', 'CH70_80',
       'CH80_90', 'CH90_00', 'CH00_10', 'POP70SQ', 'POP80SQ', 'POP90SQ',
       'POP00SQ', 'POP10SQ', 'CHSQ70_80', 'CHSQ80_90', 'CHSQ90_00',
       'CHSQ00_10', 'MILES_SQ', 'Forecast', 'ABBREVTN', 'ABBREVTN2',
       'AGENCY_CONT', 'CONTACT', 'LAST_UPDT', 'XY_SOURCE', 'XY_ISSUES',
       'ATT_SOURCE', 'ATT_ISSUES', 'COMMENTS', 'KYTC_District_Num',
       'Shape.area', 'Shape.len', 'geometry'],
      dtype='object')

In [8]:
# clean up the dataframe a little bit
cols_to_keep = ['NAME2', 'geometry']
gdf_aoi = gdf_aoi[cols_to_keep]
gdf_aoi.plot(
    color='green'  # you know, for the forest
)

ImportError: The matplotlib package is required for plotting in geopandas. You can install it using 'conda install -c conda-forge matplotlib' or 'pip install matplotlib'.

___

## Create a function to clip the tiles to the AOI

In [ ]:
#### Create a function to clip the tiles to the AOI
def clip_tiles_to_aoi(gdf_tiles, gdf_aoi):
    # clip the tiles to the AOI
    gdf_tiles_clipped = gpd.clip(gdf_tiles, gdf_aoi)
    return gdf_tiles_clipped

#### Clip the tiles

In [ ]:
gdf_tiles_clipped = clip_tiles_to_aoi(gdf_tiles, gdf_aoi)

# print the number of tiles
print(f'The clipped tiles AOI has {gdf_tiles_clipped.shape[0]} tiles.')
gdf_tiles_clipped.head(1)

In [ ]:
gdf_tiles_clipped.plot()

___
## Download tiles for AOI

Now that we've clipped the tile in our area of interest, now we want to download the files.  Here are some steps:
1. Setup boto3 for no-sign requests.
2. Choose an output location (local drive, your own AWS bucket, etc)
3. Define a function
4. Download

### Set up AWS

The bucket name is `kyfromabove`.  We just need to set that variable once.

When we set up the downloads, we need to use the `key` column from the clipped GeoDataFrame.

*Note: Refer to the Help Documentation to set up [File Transfer Configurations](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3.html)*

In [ ]:
# set up the AWS Client
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

bucket = 'kyfromabove'

# print the key values 
print(gdf_tiles_clipped['key'].head(3))

### Create a function to download files

This will leverage Geopandas *.itertuples():* to iterate through the dataframe download the files.

If you want to be prompted to enter your output folder path, uncomment this the block below.  Otherwise, skip.

In [12]:
# # enter a download path
# download_path = input('enter your output folder path: ')

# print(download_path)

If you just want to enter your path for the variable, use this block below.  Otherwise comment out or skip

In [ ]:
# enter a download path
download_path = '../downloads/tiles'  # enter your path in quotations
print(download_path)

This next block sets a limit on number of downloads.  Skip or comment it out if you want to proceed to downloading all files.

In [14]:
# create a function to download the tiles
def download_tiles(s3, bucket, gdf_tiles_clipped, download_path, max_downloads=int):  # Just downloading 5 for my example
    downloaded_files = []  # creating a list of downloaded files
    try:
        for i, row in enumerate(gdf_tiles_clipped.itertuples()):
            if i >= max_downloads:
                break  # stop after max iterations
            
            key = row.key
            # get the base file name of the key
            file_name = key.split('/')[-1]
            local_file_path = os.path.join(download_path, file_name)
            # append to file list
            downloaded_files.append(file_name)
            
            # download file
            if not os.path.exists(local_file_path):  # this will skip downloading duplicates
                s3.download_file(bucket, key, local_file_path)
                print(f'Downloaded {file_name}')
    except Exception as e:
        print(f"Error occurred: {e}")


    return downloaded_files

Uncomment this next block to download all files

In [15]:
# def download_tiles(s3, bucket, gdf_tiles_clipped, download_path):
#     downloaded_files = []  # creating a list of downloaded files
#     try:
#         for i, row in enumerate(gdf_tiles_clipped.itertuples()):
#             if i >= max_downloads:
#                 break  # stop after max iterations
            
#             key = row.key
#             # get the base file name of the key
#             file_name = key.split('/')[-1]
#             local_file_path = os.path.join(download_path, file_name)
#             # append to file list
#             downloaded_files.append(file_name)
            
#             # download file
#             if not os.path.exists(local_file_path):  # this will skip downloading duplicates
#                 s3.download_file(bucket, key, local_file_path)
#                 print(f'Downloaded {file_name}')
#     except Exception as e:
#         print(f"Error occurred: {e}")


#     return downloaded_files

#### Download files

In [ ]:
# download 30 files
download = download_tiles(s3, bucket, gdf_tiles_clipped, download_path, max_downloads=30)

My test download.<br>
<br><img src='https://raw.githubusercontent.com/ianhorn/kyfromabove-on-aws-examples/refs/heads/main/media/downloaded_tiles.jpg' width="300" height="200">

In [17]:
# # download all files
# download_tiles(s3, bucket, gdf_tiles_clipped, download_path)

___
## Create a raster or mosaic dataset in your GIS software of choice.